In [13]:
import numpy as np
import pandas as pd
# from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import recall_score, make_scorer, roc_curve
import pickle

In [1]:
# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display

%matplotlib inline

# Display full content
# pd.set_option('display.max_colwidth', -1)
# np.set_printoptions(threshold=sys.maxsize)

# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [2]:
# Project libraries
%reload_ext autoreload
from NLP import WordBag

# Read the data

In [4]:
path = './data/'
subjective_file = 'subj_review.txt'

In [5]:
f = open(path + subjective_file, 'r', encoding='utf-8', errors='ignore')
subjective_lines = []
for line in f:
    subjective_lines.append(line)
f.close()

print('Subjective:',len(subjective_lines))
subjective_lines[0]

Subjective: 5060


'called " an elegant documentary " by sundance and " eloquent and deeply moving " by the la times , toyo miyatake : infinite shades of gray is a penetrating portrait of this photographer\'s search for truth and beauty in a world of impermanence . \n'

In [6]:
objective_file = 'obj_plot.txt'
f = open(path + objective_file, 'r', encoding='utf-8', errors='ignore')
objective_lines = []
for line in f:
    objective_lines.append(line.rstrip())
f.close()

print('Objective:',len(objective_lines))

Objective: 4941


In [7]:
objective_lines[0]

'while the now 72-year-old robert evans been slowed down by a stroke , he has at least one more story to tell : his own .'

In [23]:
word_bag = WordBag()
if False:  # CV score 0.88
    all_lines =  [word_bag.clean_text(sentence, remove_stop_words=False,remove_accents=True)
                  for sentence in (subjective_lines + objective_lines)]
else:
    all_lines =  [word_bag.remove_accents(sentence)
              for sentence in (subjective_lines + objective_lines)]
print(len(all_lines))
all_lines[:50]

10001


['called " an elegant documentary " by sundance and " eloquent and deeply moving " by the la times , toyo miyatake : infinite shades of gray is a penetrating portrait of this photographer\'s search for truth and beauty in a world of impermanence . \n',
 'their reporting becomes more compelling when they gather the courage to tell the truth about witnessing two of their peers throw a four-year-old out of a 14th story window . \n',
 "calvin's barbershop is filled with an eclectic and hilarious cast of characters that share their stories , jokes , trials and tribulations . \n",
 'simultaneously devastating and hilarious , the film manages to capture the daily rituals and mundanities of life in such engrossing proximity that everything becomes drama . \n',
 'in the hilarious comedy daddy day care , two fathers ( eddie murphy , jeff garlin ) lose their jobs in product development at a large food company and are forced to take their sons out of the exclusive chapman academy and become stay-a

## Create tf_idf = f(sentence,term)

In [24]:
tfidf = TfidfVectorizer() 
sentence_tfidf = tfidf.fit_transform(all_lines)
vocab = tfidf.vocabulary_
if False:
    tfidf_mat = sentence_tfidf
else:
    tfidf_mat = sentence_tfidf.todense()
print(type(tfidf_mat))
print(len(vocab))

<class 'numpy.matrix'>
20891


In [14]:
if True:
    pickle.dump(tfidf, open('fit_tfidf_vectorizer_for_obj_subj_sentences_classification.pkl', 'wb'))

In [ ]:
#instead of default TFiDF, try:
# where 
if False:
    word_bag = WordBag()
    def dummy_fun(doc):
        return doc

    tfidf = TfidfVectorizer(
        analyzer='word',       # Feed a list of words to TF-IDF
        tokenizer=word_bag.comment_to_bag_of_words,
        preprocessor=dummy_fun,
        token_pattern=None,
        lowercase=False, 
        stop_words=None, 
        max_features=MAX_FEATURES,
        norm='l2',            # normalize each review
        use_idf=True)

In [25]:
sorted(vocab)[258:268]

['93',
 '94',
 '95',
 '996',
 '_boogie',
 'aaa',
 'aaliyah',
 'abandon',
 'abandone',
 'abandoned']

In [15]:
tfidf_mat[:2,270:300]

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

## Create Train and CV sets

In [26]:
labels = np.concatenate((np.zeros((len(subjective_lines),), dtype=int), np.ones((len(objective_lines),), dtype=int)))
print(labels.shape)
labels[4990:5010]

(10001,)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(
    tfidf_mat, labels, test_size=0.2, random_state=0)

In [28]:
print(X_train.shape, X_cv.shape, y_train.shape, y_cv.shape)

(8000, 20891) (2001, 20891) (8000,) (2001,)


## Gradient Boosting

In [29]:
# N_TREES = 300
# LEARN_RATE = 0.2
# MIN_IN_LEAF = 10
# CV score: 0.8285

# 107 trees
# LEARN_RATE = 0.2
# MAX_DEPTH = 8
# MIN_IN_LEAF = 5 #7
# MAX_FEATURES = 'sqrt'
# CV score: 0.8415

# Gradient Boosting Classifier parameters
# N_TREES = int(round(np.sqrt(X_train.shape[0]) * 1.2))
# CV score should be 0.8815
N_TREES = 300
LEARN_RATE = 0.5
MAX_DEPTH = 16
MIN_IN_LEAF = 5 #7
MAX_FEATURES = 'sqrt'
N_TREES

300

In [30]:
print(y_train[0])
print(X_train[0])


0
[[0. 0. 0. ... 0. 0. 0.]]


In [31]:
from sklearn.ensemble import GradientBoostingClassifier

# loss: deviance: logistic log likelihood
gbc = GradientBoostingClassifier(learning_rate=LEARN_RATE, 
                                   n_estimators=N_TREES, 
                                   min_samples_leaf=MIN_IN_LEAF,
                                   max_features=MAX_FEATURES)
gbc.fit(X_train, y_train)
print('Train score:', gbc.score(X_train, y_train))
print('CV score:', gbc.score(X_cv, y_cv))

Train score: 0.963125
CV score: 0.8825587206396801


In [ ]:
# save the model
import pickle
pickle.dump(gbc, open('GBC_'+ str(N_TREES) +'_' + str(LEARN_RATE) 
                        +'_' + str(MIN_IN_LEAF) + '_0.9cv.pkl', 'wb'))


# Inspect proba

In [ ]:
label_pred = gbc.predict_proba(tfidf_mat)

In [ ]:
print(label_pred[:10,1])
print(labels[:10])

In [ ]:
print(label_pred[-10:,1])
print(labels[-10:])

In [ ]:
fpr, tpr, thresholds = roc_curve(labels, label_pred[:,1])
plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate (1 - Specificity)")
plt.ylabel("True Positive Rate (Sensitivity, Recall)")
plt.title("ROC plot")

In [ ]:
np.argmin(fpr > 0)
print(fpr[:20])

In [ ]:
t_f = tpr[5:]/fpr[5:]
# plt.plot(range(len(t_f)), t_f)
plt.plot(np.flip(thresholds[5:]), t_f)
# plt.plot(thresholds[5:], t_f)
plt.xlabel("Threshold")
plt.ylabel("True Positive Rate / False Positive Rate")
plt.xlim([0,0.5])
plt.ylim([0,400])
plt.title("TRP/FPR = f(threshold)")

## Some False Positives examples: predicted objective, actual subjective

In [ ]:
THRESHOLD = 0.8
all_lines_arr = np.array(all_lines)

In [ ]:
TP = all_lines_arr[(label_pred[:,1] > THRESHOLD) & (labels == 1)]
FP = all_lines_arr[(label_pred[:,1] > THRESHOLD) & (labels == 0)]
TP_nb = len(TP)
FP_nb = len(FP)
TPR = TP_nb / len([labels == 1])
FPR = FP_nb / len([labels == 0])

predicted_P = FP_nb + TP_nb
if predicted_P != len(all_lines_arr[label_pred[:,1] > THRESHOLD]):
    print('Problem!')

print('FPR = {}'.format(FPR))
display(Markdown('### Removing {} sentences assumed objective'.format(predicted_P, )))
display(Markdown('### {0} ({1:.1%}) falsely predicted objective:'.format(FP_nb, FP_nb / predicted_P)))
for i, sent in enumerate(FP):
    print(i, sent)

# Predicted subjective, actually objective

In [ ]:
diff = np.array(all_lines)[(label_pred == 1) & (labels == 0)]
print(len(diff))

for i, sent in enumerate(diff):
    print(i, sent)

# Grid search

In [ ]:
if True:
    grid = {
        'learning_rate': [.01, .05],
        'max_depth': [8, 16],
        'min_samples_leaf': [5],
        'max_features': ['sqrt'],
        'n_estimators': [300],
        'random_state': [0],
    }
else:  # TEST
    grid = {
    'learning_rate': [1],
    'max_depth': [2], 
    'min_samples_leaf': [2],
#     'max_features': ['sqrt', None],
    'n_estimators': [2],
    'random_state': [0]
}
    
# confusion_score = make_scorer(confusion_rmse, greater_is_better=False)

scorer = make_scorer(recall_score)
gbc_grid_cv = GridSearchCV(
    GradientBoostingClassifier(), 
    grid,
    cv=4,  # number of folds
    return_train_score=True,
    scoring=scorer,
    verbose=1, 
    n_jobs=-1)
gbc_grid_cv.fit(X_train, y_train)

In [ ]:
y_pred = gbc_grid_cv.predict(X_cv)

print(gbc_grid_cv.best_params_)
print(gbc_grid_cv.best_score_)
res_df = pd.DataFrame(gbc_grid_cv.cv_results_)
print('Used recall for scoring')
res_df

# LSTM

In [ ]:
print(len(all_lines))
all_lines[0]

### Find the max length of comments

In [ ]:
all_lines[3312:3322]

In [ ]:
cleaned_text[3312:3322]

In [ ]:
if False:
    for i, text in enumerate(cleaned_text):
        if len(text.split()) > 60:
            print(i, text)

In [ ]:
if False:
    for i, text in enumerate(all_lines):
        if len(text.split()) > 60:
            print(i, text)

In [ ]:
max([len(text.split()) for text in all_lines])

## Text to sequence

In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(
    all_lines, labels, test_size=0.2, random_state=0)

### Define & run LSTM

In [ ]:
WORD_WINDOW = 50

import tensorflow as tf
keras = tf.keras
lstm = keras.Sequential()
lstm.add(keras.layers.LSTM(32, input_shape=(WORD_WINDOW, 1), return_sequences=True))
lstm.add(keras.layers.LSTM(32, return_sequences=False))
lstm.add(keras.layers.Dense(1, activation='linear'))
lstm.compile(optimizer='rmsprop',
              loss='mse')

In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(
    all_lines, labels, test_size=0.2, random_state=0)

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=10)

## J

In [ ]:
MAX_NB_WORDS = 500
max_review_length = 500
EMBEDDING_DIM = 160

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = MAX_NB_WORDS, 
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                      lower=True, split=' ', char_level=False, 
                      oov_token=None, document_count=0)

tokenizer.fit_on_texts(all_lines)

In [ ]:
config = tokenizer.get_config()

In [ ]:
config['word_counts']

In [ ]:
train_sequences = tokenizer.texts_to_sequences(df_tem3['review'])
test_sequences = tokenizer.texts_to_sequences(test['review'])

## TODO
### Use emotions vectors as input to model
### Use emotions to inspect input and re-label if needed
### See if remove highest frequency words?